# Synchronize homework files

In [ ]:
! rm -rf sample_data; curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/sync_hw.py > sync_hw.py
files = ["quadrotor.py", "quadrotor_robber.py", "quadrotor_centralized.py", "create_animation.py", "quad_sim.py",
         "quad_sim_centralized.py", "environment_manager.py", "initial_states.py"]

from sync_hw import sync_hw
sync_hw(6, files, True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Drake and set up a server for the visualizer

In [ ]:
# Install drake. We are using the lqr controller in drake as the final stabilizing controller.
# The installation process will take about 2 minutes but it's only required in the start of the Colab's virtual machine.
!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()

<IPython.core.display.Javascript object>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
# import matplotlib; matplotlib.use("TkAgg")
import matplotlib.animation as animation

# Import Simulator
from quad_sim import simulate_quadrotor
from quad_sim_centralized import simulate_quadrotor_centralized
import initial_states

# Load Quadrotor Objects
import quadrotor
import quadrotor_centralized
import quadrotor_robber

# Reload Modules to Use Latest Code
importlib.reload(quadrotor)
importlib.reload(quadrotor_centralized)
importlib.reload(quadrotor_robber)
from quadrotor import Quadrotor
from quadrotor_centralized import QuadrotorCentralized
from quadrotor_robber import QuadrotorRobber

# Load Environment Manager
import environment_manager
importlib.reload(environment_manager)
from environment_manager import Map

# Load Animation Function
import create_animation
importlib.reload(create_animation)
from create_animation import create_animation

# Bounds of Environment
y_min = 0
y_max = 40
z_min = 0
z_max = 12.5

# Map
ax = plt.axes()
plt.close()
environment = Map()  # Initialize Map Class
# obstacles = environment.map_3() # Fixed
obstacles = environment.map_4(ax, y_min, y_max, z_min, z_max) # Random
# obstacles = environment.map_5() # Fixed Dense Maze
# obstacles = environment.map_6() # Fixed Sparse, Small Obstacle Maze
# obstacles = environment.map_7() # Fixed Sparse, Large Obstacle Maze
# obstacles = environment.map_8() # City Blocks, Gaps
# obstacles = environment.map_9() # City Blocks, Solid Walls

# Weights of LQR cost
R = np.eye(2)
Q = np.diag([1, 1, 0, 1, 1, 1])
Qf = Q

# End time of the simulation
tf = 30

# Number of Cop Quadrotors
num_cops = 1

# Construct Cop Quadrotor Controllers
# Distributed MPC Quadrotor Objects
quad_cops = []
for i in range(num_cops):
  quad_cops.append(Quadrotor(Q, R, Qf))

# Centralized Quadrotor Objects
quadrotor_central = QuadrotorCentralized(Q, R, Qf)

# Construct Robber Quadrotor Controller
quad_robber = QuadrotorRobber(Q, R, Qf)

# Initial States of Quadrotors
# Fixed Initializations
# x0_cops = initial_states.cops_fixed(num_cops) # Fixed Cop Initializations
# x0_robber = initial_states.robber_fixed() # Fixed Robber Initialization
# Random Initializations
x0_cops = initial_states.cops_random(obstacles, num_cops, y_min, y_max, z_min, z_max) # Random Cop Initializations
x0_robber = initial_states.robber_random(x0_cops, obstacles, num_cops, y_min, y_max, z_min, z_max) # Random Robber Initialization
x0_robber_des = initial_states.init_robber_des(obstacles, y_min, y_max, z_min, z_max)

# Simulate Quadrotors
# Distributed MPC
x_cops, x_cop_d, u_cops, x_robber, x_rob_d, u_robber, t, t_e = simulate_quadrotor(x0_cops, x0_robber, quad_cops, quad_robber,
                                                                             tf, num_cops, obstacles, x0_robber_des)
# Create Animation
x_out = np.stack(([c for c in x_cops] + [x_robber]), axis=0)
x_d_out = np.stack((x_cop_d, x_rob_d), axis=0)
anim, fig2 = create_animation(x_out, x_d_out, t, obstacles, num_cops + 1, title="Distributed")
# writervideo = animation.FFMpegWriter(fps=5)
# anim.save('distributed.mp4',writer=writervideo)
plt.close()
anim


Generated Obstacles
outside_while 1
Initialized Cops
outside_while 1
Initialized Robber
time: 0.01
time: 0.02
time: 0.03
time: 0.04
time: 0.05
time: 0.060000000000000005
time: 0.07
time: 0.08
time: 0.09
time: 0.09999999999999999
time: 0.10999999999999999
time: 0.11999999999999998
time: 0.12999999999999998
time: 0.13999999999999999
time: 0.15
time: 0.16
time: 0.17
time: 0.18000000000000002
time: 0.19000000000000003
time: 0.20000000000000004
time: 0.21000000000000005
time: 0.22000000000000006
time: 0.23000000000000007
time: 0.24000000000000007
time: 0.25000000000000006
time: 0.26000000000000006
time: 0.2700000000000001
time: 0.2800000000000001
time: 0.2900000000000001
time: 0.3000000000000001
time: 0.3100000000000001
time: 0.3200000000000001
time: 0.3300000000000001
time: 0.34000000000000014
time: 0.35000000000000014
time: 0.36000000000000015
time: 0.37000000000000016
time: 0.38000000000000017
time: 0.3900000000000002
time: 0.4000000000000002
time: 0.4100000000000002
time: 0.420000000000

In [ ]:
# Simulate Quadrotors
# Centralized MPC
x_cops_c, x_cop_d_c, u_cops_c, x_robber_c, x_rob_d_c, u_robber_c, t_c, t_e_c = simulate_quadrotor_centralized(x0_cops, x0_robber, quadrotor_central,
                                                                                         quad_robber, tf, num_cops, obstacles, x0_robber_des)

# Create Animation
x_out_c = np.stack(([c for c in x_cops_c] + [x_robber_c]), axis=0)
x_d_out_c = np.stack((x_cop_d_c, x_rob_d_c), axis=0)

print("\nDistributed: t_f = {:.2f}, t_elapsed = {:.2f}".format(t[-1], t_e))
print("Centralized: t_f = {:.2f}, t_elapsed = {:.2f}".format(t_c[-1], t_e_c))
anim_c, fig3 = create_animation(x_out_c, x_d_out_c, t_c, obstacles, num_cops + 1, title = "Centralized")
# anim_c.save('central.mp4',writer=writervideo)
plt.close()
anim_c

time: 0.01
desired: [11.60767252  3.16982774  0.          0.          0.          0.        ]

time: 0.02
desired: [11.60767252  3.16982774  0.          0.          0.          0.        ]

time: 0.03
desired: [11.60767252  3.16981712  0.          0.          0.          0.        ]

time: 0.04
desired: [11.60767252  3.16980747  0.          0.          0.          0.        ]

time: 0.05
desired: [11.60767304  3.16980962  0.          0.          0.          0.        ]

time: 0.060000000000000005
desired: [11.60767507  3.16983366  0.          0.          0.          0.        ]

time: 0.07
desired: [11.60768006  3.16988899  0.          0.          0.          0.        ]

time: 0.08
desired: [11.60768989  3.16998436  0.          0.          0.          0.        ]

time: 0.09
desired: [11.60770681  3.17012793  0.          0.          0.          0.        ]

time: 0.09999999999999999
desired: [11.60773345  3.17032726  0.          0.          0.          0.        ]

time: 0.10999999999